In [1]:
import pandas as pd
import numpy as np

path = '/Users/purchasedinukraine/Downloads/data_preproc.csv'

df = pd.read_csv(path)

In [12]:
single_unique_cols = [col for col in df.columns if df[col].nunique() == 1]
df.drop(columns=['Flow.ID', 'Timestamp'] + single_unique_cols, inplace=True)
list(df.columns)

['Source.IP',
 'Source.Port',
 'Destination.IP',
 'Destination.Port',
 'Protocol',
 'Flow.Duration',
 'Total.Fwd.Packets',
 'Total.Backward.Packets',
 'Total.Length.of.Fwd.Packets',
 'Total.Length.of.Bwd.Packets',
 'Fwd.Packet.Length.Max',
 'Fwd.Packet.Length.Min',
 'Fwd.Packet.Length.Mean',
 'Fwd.Packet.Length.Std',
 'Bwd.Packet.Length.Max',
 'Bwd.Packet.Length.Min',
 'Bwd.Packet.Length.Mean',
 'Bwd.Packet.Length.Std',
 'Flow.Bytes.s',
 'Flow.Packets.s',
 'Flow.IAT.Mean',
 'Flow.IAT.Std',
 'Flow.IAT.Max',
 'Flow.IAT.Min',
 'Fwd.IAT.Total',
 'Fwd.IAT.Mean',
 'Fwd.IAT.Std',
 'Fwd.IAT.Max',
 'Fwd.IAT.Min',
 'Bwd.IAT.Total',
 'Bwd.IAT.Mean',
 'Bwd.IAT.Std',
 'Bwd.IAT.Max',
 'Bwd.IAT.Min',
 'Fwd.PSH.Flags',
 'Fwd.Header.Length',
 'Bwd.Header.Length',
 'Fwd.Packets.s',
 'Bwd.Packets.s',
 'Min.Packet.Length',
 'Max.Packet.Length',
 'Packet.Length.Mean',
 'Packet.Length.Std',
 'Packet.Length.Variance',
 'FIN.Flag.Count',
 'SYN.Flag.Count',
 'RST.Flag.Count',
 'PSH.Flag.Count',
 'ACK.Flag.Coun

In [8]:
freq_encoding = (df.groupby('Source.IP').size()) / len(df)
df['Source.IP.FreqEnc'] = df['Source.IP'].apply(lambda x: freq_encoding[x])
df.drop(columns=['Source.IP'], inplace=True)
print('done')

done


In [14]:
_tmp = pd.get_dummies(df['Protocol']).rename(columns={0:"Protocol:Unknown", 6:"Protocol.TCP", 17:"Protocol.UDP"})
df = pd.concat([df, _tmp], axis=1)


In [15]:
df.drop(columns=["Protocol:Unknown", "Protocol.TCP", "Protocol.UDP"], inplace=True)
df = pd.concat([df, _tmp], axis=1)

In [2]:
print(df.head(10))


       Source.IP  Source.Port Destination.IP  Destination.Port  Protocol  \
0    172.19.1.46        52422     10.200.7.7              3128         6   
1     10.200.7.7         3128    172.19.1.46             52422         6   
2   50.31.185.39           80   10.200.7.217             38848         6   
3   50.31.185.39           80   10.200.7.217             38848         6   
4  192.168.72.43        55961     10.200.7.7              3128         6   
5     10.200.7.6         3128    172.19.1.56             50004         6   
6  192.168.72.43        55963     10.200.7.7              3128         6   
7  192.168.10.47        51848     10.200.7.6              3128         6   
8  68.67.178.197          443   10.200.7.217             57300         6   
9  192.168.72.43        55977     10.200.7.7              3128         6   

   Flow.Duration  Total.Fwd.Packets  Total.Backward.Packets  \
0          45523                 22                      55   
1              1                  2  

In [16]:
df.drop(columns=['Protocol', 'Active.Min', 'Idle.Min', 'Min.Packet.Length', 'Bwd.IAT.Min', 'Fwd.IAT.Min',
                 'Flow.IAT.Min', 'Bwd.Packet.Length.Min', 'Fwd.Packet.Length.Min'], inplace=True)
df.head(3)

,Source.IP,Source.Port,Destination.IP,Destination.Port,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,Total.Length.of.Fwd.Packets,Total.Length.of.Bwd.Packets,Fwd.Packet.Length.Max,...,Active.Std,Active.Max,Idle.Mean,Idle.Std,Idle.Max,L7Protocol,ProtocolName.FreqEnc,Protocol:Unknown,Protocol.TCP,Protocol.UDP
0,172.19.1.46,52422,10.200.7.7,3128,45523,22,55,132,110414.0,6,...,0.0,0.0,0.0,0.0,0.0,131,0.174213,0,1,0
1,10.200.7.7,3128,172.19.1.46,52422,1,2,0,12,0.0,6,...,0.0,0.0,0.0,0.0,0.0,131,0.174213,0,1,0
2,50.31.185.39,80,10.200.7.217,38848,1,3,0,674,0.0,337,...,0.0,0.0,0.0,0.0,0.0,7,0.191132,0,1,0


In [12]:
df.drop(columns=["L7Protocol"], inplace=True)

In [13]:
df.to_csv(path)
print('done!')

done!


In [2]:
def normalizz(col):
    df[col] = (df[col]-df[col].min())/(df[col].max()-df[col].min())

In [10]:
forbid = ['Source.IP', 'Destination.IP', 'Source.Port', 'Destination.Port', 'Protocol:Unknown',
       'Protocol.TCP', 'Protocol.UDP',  'L7Protocol', 'ProtocolName.FreqEnc', 'Fwd.PSH.Flags']
for a in df.columns:
    if a not in forbid:
        df[a] = (df[a]-df[a].min())/(df[a].max()-df[a].min())
        print('Done: '+ a)

Done: Unnamed: 0
Done: Flow.Duration
Done: Total.Fwd.Packets
Done: Total.Backward.Packets
Done: Total.Length.of.Fwd.Packets
Done: Total.Length.of.Bwd.Packets
Done: Fwd.Packet.Length.Max
Done: Fwd.Packet.Length.Mean
Done: Fwd.Packet.Length.Std
Done: Bwd.Packet.Length.Max
Done: Bwd.Packet.Length.Mean
Done: Bwd.Packet.Length.Std
Done: Flow.Bytes.s
Done: Flow.Packets.s
Done: Flow.IAT.Mean
Done: Flow.IAT.Std
Done: Flow.IAT.Max
Done: Fwd.IAT.Total
Done: Fwd.IAT.Mean
Done: Fwd.IAT.Std
Done: Fwd.IAT.Max
Done: Bwd.IAT.Total
Done: Bwd.IAT.Mean
Done: Bwd.IAT.Std
Done: Bwd.IAT.Max
Done: Fwd.Header.Length
Done: Bwd.Header.Length
Done: Fwd.Packets.s
Done: Bwd.Packets.s
Done: Max.Packet.Length
Done: Packet.Length.Mean
Done: Packet.Length.Std
Done: Packet.Length.Variance
Done: FIN.Flag.Count
Done: SYN.Flag.Count
Done: RST.Flag.Count
Done: PSH.Flag.Count
Done: ACK.Flag.Count
Done: URG.Flag.Count
Done: ECE.Flag.Count
Done: Down.Up.Ratio
Done: Average.Packet.Size
Done: Avg.Fwd.Segment.Size
Done: Avg.Bwd.